In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os
from numpy import nan as NA
import geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")

## Check for inconsistencies across monthly datasets

When we manually checked for how data files were structured, we found that there are some inconsistencies across our individual datasets. They seemed to be difference before/after covid, so we had to check for this.

In [4]:
dfn = pd.read_csv('/Users/jennyjang/Desktop/CS Project/datasets/202003-capitalbikeshare-tripdata.csv')
dfo = pd.read_csv('/Users/jennyjang/Desktop/CS Project/datasets/202005-capitalbikeshare-tripdata.csv')

In [5]:
#Column headings differ between March 2020 vs. May 2020 so we checked to see 
#how else the files differed in terms of format.

#membership type
print("membership type\n")
print("March 2020: ", dfn['Member type'].value_counts())
print("May 2020: ", dfo['member_casual'].value_counts())  #capitalization differs

#Dates
print("\n dates\n")
print("March 2020: ", dfn['Start date'].head())
print("May 2020: ", dfo['started_at'].tail())  #no difference between the two time periods.

#Ride id / bike number
print("\n bike id\n")
print("March 2020: ", dfn['Bike number'].head())
print("May 2020: ", dfo['ride_id'].head())   # very different format but of no consequence for analysis.

#Station number
print("\n station id\n")
print("March 2020: ", dfn['Start station number'].head())
print("May 2020: ", dfo['start_station_id'].tail())   # No apparent difference

#Station address / name
print("\n station name\n")
print("March 2020: ", dfn['Start station'].tail())
print("May 2020: ", dfo['start_station_name'].tail())   # No apparent difference

membership type

March 2020:  Member    145923
Casual     16607
Name: Member type, dtype: int64
May 2020:  casual    75546
member    73505
Name: member_casual, dtype: int64

 dates

March 2020:  0    2020-03-01 00:01:16
1    2020-03-01 00:02:04
2    2020-03-01 00:03:45
3    2020-03-01 00:05:49
4    2020-03-01 00:06:53
Name: Start date, dtype: object
May 2020:  149046    2020-05-12 08:15:31
149047    2020-05-17 13:19:33
149048    2020-05-30 14:53:00
149049    2020-05-27 17:09:42
149050    2020-05-15 15:02:35
Name: started_at, dtype: object

 bike id

March 2020:  0    W24217
1    W21897
2    W21703
3     78571
4    W23425
Name: Bike number, dtype: object
May 2020:  0    946D42AD89539210
1    CC46FAAB662B8613
2    72F00B2FB833D6ED
3    4DFBE6AED989DF35
4    1AAFE6B4331AB9DF
Name: ride_id, dtype: object

 station id

March 2020:  0    31646
1    31646
2    31224
3    31603
4    31101
Name: Start station number, dtype: int64
May 2020:  149046    31229
149047    31229
149048    31002
149049

After this, we were able to figure out how we would want to concatenate all our datasets.

## Concatenating all our datasets

The next step was to bring our dataset together.
There were one file name without csv, so wrote a forloop to account for that. We also added a variable 'fileid' to keep track of data in the beginning. 
And printed columns and check the shape of our dataframe.

In [2]:
list_of_df = []
for file in os.listdir("/Users/jennyjang/Desktop/CS Project/datasets/"):
    if file != '.DS_Store':
        if '.csv' in file:
            df = pd.read_csv(os.path.join("/Users/jennyjang/Desktop/CS Project/datasets", file))
        else:
            df = pd.read_csv(os.path.join("/Users/jennyjang/Desktop/CS Project/datasets", file), sep=",")
        df.rename(columns = {'Start date':'started_at', 
                              'End date':'ended_at',
                              'Start station':'start_station_name',
                              'Start station number':'start_station_id',
                              'End station number':'end_station_id',
                              'End station':'end_station_name',
                              'Bike number':'ride_id',
                              'Member type':'member_casual'
                              }, inplace = True)
        df['fileid']=df['started_at'].str[0:7]
        df['member_casual']=df['member_casual'].str.lower()
        list_of_df.append(df)
df = pd.concat(list_of_df) #Puts all DF together

df["id"] = df.index + 1 #Create unique stable id for each observation
print(df.columns)
print(df.shape)

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Index(['Duration', 'started_at', 'ended_at', 'start_station_id',
       'start_station_name', 'end_station_id', 'end_station_name', 'ride_id',
       'member_casual', 'fileid', 'rideable_type', 'start_lat', 'start_lng',
       'end_lat', 'end_lng', 'id'],
      dtype='object')
(5737820, 16)


After all the datasets were concatenated,  it has 5737820 rows and 16 columns.

## Checking for duplicates and missing values

In [ ]:
dup=df.duplicated()
print("There are", dup.sum(),"duplicate records.")
#Conclusion: No dups

In [ ]:

num2=0 #Check number and percent of missings for each column
for f in frames:  
  print(num2)
  for i in f.columns:
    print(i,":",f[i].isnull().sum(),round(f[i].isnull().sum()/f.shape[0],5) )
  num2=num2+1

Virtually no missing values before March 2020.  
After March 2020, station names and ids are increasingly missing. In these
cases, lat-long is almost always available. To fill in missing data about 
about location we will need to impute based on latitude and longitude to know the location.

## Filling out missing longitude and latitude

Datasets before covid have less variables than datasets after covid. We wanted to use longitude and latitude to compare some aspects, so decided to fill in missing vales for 'long' and 'lat'. 
This was very challenging as our data contains a lot of inconsistencies and missing values. 

In [3]:
df = df[df["start_station_id"] != 'MTL-ECO5-03']
temp = df[["start_station_id","start_lat","start_lng"]]
df_start_lat_lng = temp[temp["start_lat"] != 'nan']
df_start_lat_lng["start_station_id"] = df_start_lat_lng["start_station_id"].astype(float).astype(str)
df_start_lat_lng["start_station_id"] = df_start_lat_lng["start_station_id"].str.split('.').str[0]
df_start_lat_lng = df_start_lat_lng.groupby("start_station_id").mean().reset_index()

df = df[df["end_station_id"] != 'MTL-ECO5-03']
temp = df[["end_station_id","end_lat","end_lng"]]
df_end_lat_lng = temp[temp["end_lat"] != 'nan']
df_end_lat_lng["end_station_id"] = df_end_lat_lng["end_station_id"].astype(float).astype(str)
df_end_lat_lng["end_station_id"] = df_end_lat_lng["end_station_id"].str.split('.').str[0]
df_end_lat_lng = df_end_lat_lng.groupby("end_station_id").mean().reset_index()

There was a random station id, so we tookout that station id.
Then we created a temperary dataset so that we can create a subdataset with just the station id and lat/long for that station.
As there were 3 types of vlaues in 'station id' column, we had to make it conherent in order to do groupby. 
Because the lat/long were slightly different even for same stations, we just calculated mean of lat/long to make things consistent and be able to use groupby.

To start filling out the missing vlaues, we created a dictionary with 'station id' as key and 'lat','long' as our values

In [4]:
dict_start_lat_lng = df_start_lat_lng.set_index('start_station_id').T.to_dict('list')
dict_end_lat_lng = df_end_lat_lng.set_index('end_station_id').T.to_dict('list')
# dict_start_lat_lng
# dict_end_lat_lng

Then we made definitions that would go through rows in our data frame and check if the lat/long is missing and if it is, return the lat/long that coressponds with the station id and if it is not missing, then find the corresponding lat/long looking up the station id. Some of our station ids were in float, so we split it with '.' and took the first part to make it standardized.

In [5]:
def fillLatNa(row):
    if row["start_station_id"] == 'nan':
        return [row["start_lat"], row["start_lng"]]
    elif str(row["start_station_id"]).split('.')[0] in dict_start_lat_lng.keys():
        return dict_start_lat_lng[str(row["start_station_id"]).split('.')[0]]
    else:
        print(row)

lat_long_apply = df.apply(fillLatNa, axis=1)
lat_long_apply

def fillLatNaEnd(row):
    if row["end_station_id"] == 'nan':
        return [row["end_lat"], row["end_lng"]]
    elif str(row["end_station_id"]).split('.')[0] in dict_end_lat_lng.keys():
        return dict_end_lat_lng[str(row["end_station_id"]).split('.')[0]]
    else:
        print(row)

lat_long_end_apply = df.apply(fillLatNaEnd, axis=1)
lat_long_end_apply

0          [38.90859960233324, -77.03230730090273]
1         [38.887375235293405, -77.00195490068289]
2         [38.903812804326385, -77.02841438144965]
3          [38.958277258122955, -77.0846325542557]
4          [38.91549878955074, -77.02220440052463]
                            ...                   
158125     [38.89476082707194, -76.99712444206192]
158126    [38.905736689850286, -77.02226775031674]
158127     [38.903042880025154, -77.0190324735292]
158128     [38.90240127054438, -77.01600530816451]
158129     [38.90207047347375, -77.03833115016533]
Length: 5737818, dtype: object

When finished, parse the list to lat and long, and make new columns, put that parsed value into the corresponding column.

In [6]:
df["temp_start_lat"] = lat_long_apply.str[0]
df["temp_start_lng"] = lat_long_apply.str[1]
df["temp_end_lat"] = lat_long_end_apply.str[0]
df["temp_end_lng"] = lat_long_end_apply.str[1]
df # Check if the columns has been created successfully.

,Duration,started_at,ended_at,start_station_id,start_station_name,end_station_id,end_station_name,ride_id,member_casual,fileid,rideable_type,start_lat,start_lng,end_lat,end_lng,id,temp_start_lat,temp_start_lng,temp_end_lat,temp_end_lng
0,429.0,2019-10-01 00:01:59,2019-10-01 00:09:08,31214,17th & Corcoran St NW,31203,14th & Rhode Island Ave NW,W23731,member,2019-10,NaN,NaN,NaN,NaN,NaN,1,38.912097,-77.038689,38.908600,-77.032307
1,1935.0,2019-10-01 00:03:07,2019-10-01 00:35:23,31269,3rd St & Pennsylvania Ave SE,31269,3rd St & Pennsylvania Ave SE,W22377,member,2019-10,NaN,NaN,NaN,NaN,NaN,2,38.887374,-77.001958,38.887375,-77.001955
2,563.0,2019-10-01 00:03:51,2019-10-01 00:13:14,31214,17th & Corcoran St NW,31251,12th & L St NW,W23473,member,2019-10,NaN,NaN,NaN,NaN,NaN,3,38.912097,-77.038689,38.903813,-77.028414
3,625.0,2019-10-01 00:05:34,2019-10-01 00:15:59,31301,Ward Circle / American University,31317,Wisconsin Ave & Ingomar St NW,W21371,member,2019-10,NaN,NaN,NaN,NaN,NaN,4,38.938741,-77.087181,38.958277,-77.084633
4,312.0,2019-10-01 00:05:47,2019-10-01 00:10:59,31519,1st & O St NW,31109,7th & T St NW,W23666,member,2019-10,NaN,NaN,NaN,NaN,NaN,5,38.908644,-77.012366,38.915499,-77.022204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158125,886.0,2019-02-28 23:55:21,2019-03-01 00:10:08,31223,Convention Center / 7th & M St NW,31612,D St & Maryland Ave NE,W23129,member,2019-02,NaN,NaN,NaN,NaN,NaN,158126,38.905731,-77.022264,38.894761,-76.997124
158126,404.0,2019-02-28 23:57:03,2019-03-01 00:03:47,31203,14th & Rhode Island Ave NW,31223,Convention Center / 7th & M St NW,W00316,member,2019-02,NaN,NaN,NaN,NaN,NaN,158127,38.908596,-77.032308,38.905737,-77.022268
158127,972.0,2019-02-28 23:57:03,2019-03-01 00:13:16,31101,14th & V St NW,31600,5th & K St NW,W21796,member,2019-02,NaN,NaN,NaN,NaN,NaN,158128,38.917807,-77.032079,38.903043,-77.019032
158128,925.0,2019-02-28 23:57:32,2019-03-01 00:12:58,31613,Eastern Market Metro / Pennsylvania Ave & 7th ...,31642,New Jersey Ave & L St NW,W23576,member,2019-02,NaN,NaN,NaN,NaN,NaN,158129,38.884090,-76.995180,38.902401,-77.016005


Looking at the table above, we can see that the lat/longs has been filled in correctly. We also manually looking those coordinates up to double check if the lat/long corresponds with the correct stations, and they did.


In [7]:
for col in df.columns:
    print(col)

Duration
started_at
ended_at
start_station_id
start_station_name
end_station_id
end_station_name
ride_id
member_casual
fileid
rideable_type
start_lat
start_lng
end_lat
end_lng
id
temp_start_lat
temp_start_lng
temp_end_lat
temp_end_lng


After seeing that the lat and long has been successfully added, we are going to drop the original lat/long columns and rename the new columns.

In [8]:
# renaming the new columns
df = df.drop(columns=['start_lat','start_lng','end_lat','end_lng'])
df = df.rename(columns = {'temp_start_lat': 'start_lat', 'temp_start_lng': 'start_lng', 'temp_end_lat':'end_lat', 'temp_end_lng':'end_lng' }, inplace = False)
df

,Duration,started_at,ended_at,start_station_id,start_station_name,end_station_id,end_station_name,ride_id,member_casual,fileid,rideable_type,id,start_lat,start_lng,end_lat,end_lng
0,429.0,2019-10-01 00:01:59,2019-10-01 00:09:08,31214,17th & Corcoran St NW,31203,14th & Rhode Island Ave NW,W23731,member,2019-10,NaN,1,38.912097,-77.038689,38.908600,-77.032307
1,1935.0,2019-10-01 00:03:07,2019-10-01 00:35:23,31269,3rd St & Pennsylvania Ave SE,31269,3rd St & Pennsylvania Ave SE,W22377,member,2019-10,NaN,2,38.887374,-77.001958,38.887375,-77.001955
2,563.0,2019-10-01 00:03:51,2019-10-01 00:13:14,31214,17th & Corcoran St NW,31251,12th & L St NW,W23473,member,2019-10,NaN,3,38.912097,-77.038689,38.903813,-77.028414
3,625.0,2019-10-01 00:05:34,2019-10-01 00:15:59,31301,Ward Circle / American University,31317,Wisconsin Ave & Ingomar St NW,W21371,member,2019-10,NaN,4,38.938741,-77.087181,38.958277,-77.084633
4,312.0,2019-10-01 00:05:47,2019-10-01 00:10:59,31519,1st & O St NW,31109,7th & T St NW,W23666,member,2019-10,NaN,5,38.908644,-77.012366,38.915499,-77.022204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158125,886.0,2019-02-28 23:55:21,2019-03-01 00:10:08,31223,Convention Center / 7th & M St NW,31612,D St & Maryland Ave NE,W23129,member,2019-02,NaN,158126,38.905731,-77.022264,38.894761,-76.997124
158126,404.0,2019-02-28 23:57:03,2019-03-01 00:03:47,31203,14th & Rhode Island Ave NW,31223,Convention Center / 7th & M St NW,W00316,member,2019-02,NaN,158127,38.908596,-77.032308,38.905737,-77.022268
158127,972.0,2019-02-28 23:57:03,2019-03-01 00:13:16,31101,14th & V St NW,31600,5th & K St NW,W21796,member,2019-02,NaN,158128,38.917807,-77.032079,38.903043,-77.019032
158128,925.0,2019-02-28 23:57:32,2019-03-01 00:12:58,31613,Eastern Market Metro / Pennsylvania Ave & 7th ...,31642,New Jersey Ave & L St NW,W23576,member,2019-02,NaN,158129,38.884090,-76.995180,38.902401,-77.016005


# Using GEOPY to extract 'State' Information

Now that we have lat/long informations, we are now going to use GEOPY to extract more detailed information about locations of station. We are doing this because we only want records that started in 'Virginia' as that was our main focus.

Our lat/longs are good information, but what was tricky is that the lat/longs would differ slightly even for the same station. Because of this, running geopy to go through more than 5.5M records was just not efficient. 

In order to solve this problem, we decided to aggregate on the 'Station id' and take the average of lat/longs. This method was ok since the lat/longs only differed very slightly.

Starting with creating a temporary df to work with:

In [19]:
temp = df[["start_station_id","start_lat","start_lng"]]
temp = temp.groupby("start_station_id").mean()
temp= temp[temp['start_lat'].notnull()]
temp

,start_lat,start_lng
start_station_id,,
31000,38.858972,-77.053234
31001,38.857250,-77.053355
31002,38.856422,-77.049237
31003,38.860282,-77.049568
31004,38.857866,-77.059490
...,...,...
32604,38.880605,-77.171898
32605,38.883591,-77.173437
32606,38.878086,-77.153957


We found this method from https://www.geeksforgeeks.org/get-the-city-state-and-country-names-from-latitude-and-longitude-using-python/. 

But we had to first make the lat/long into specific format for this method to work. So that is what we are doing here:

In [28]:
# https://www.geeksforgeeks.org/get-the-city-state-and-country-names-from-latitude-and-longitude-using-python/ 
startlatlst=temp['start_lat'].tolist()
startlnglst=temp['start_lng'].tolist()
newlst=[]
x=list((zip(startlatlst,startlnglst)))
for i in x:
  x0=round(i[0],6)
  x1=round(i[1],6)
  newpair=str(x0) + ","+str(x1)
  newlst.append(newpair)
print(len(newlst))

1203


Now, using the geolocator to find the state information and appending that to the a new list and turning that list into a new column in our temporary df:

In [29]:
statels=[]
for i in newlst:
  try:
    location=(geolocator.reverse(i))
    address = location.raw['address']
    state = address.get('state', '')
    statels.append(state)
  except:
    print('error in',i)
    break
temp['state']=statels

In [33]:
temp

,start_lat,start_lng,state
start_station_id,,,
31000,38.858972,-77.053234,Virginia
31001,38.857250,-77.053355,Virginia
31002,38.856422,-77.049237,Virginia
31003,38.860282,-77.049568,Virginia
31004,38.857866,-77.059490,Virginia
...,...,...,...
32604,38.880605,-77.171898,Virginia
32605,38.883591,-77.173437,Virginia
32606,38.878086,-77.153957,Virginia


We have successfully located State for each records.

Now merging this temporary dataset to our big df:

In [34]:
# Now merging two df based on the station_id
df = pd.merge(left=df, right=temp, how='left', left_on='start_station_id', right_on='start_station_id')
df

,Duration,started_at,ended_at,start_station_id,start_station_name,end_station_id,end_station_name,ride_id,member_casual,fileid,rideable_type,id,start_lat_x,start_lng_x,end_lat,end_lng,start_lat_y,start_lng_y,state
0,429.0,2019-10-01 00:01:59,2019-10-01 00:09:08,31214,17th & Corcoran St NW,31203,14th & Rhode Island Ave NW,W23731,member,2019-10,NaN,1,38.912097,-77.038689,38.908600,-77.032307,38.912097,-77.038689,District of Columbia
1,1935.0,2019-10-01 00:03:07,2019-10-01 00:35:23,31269,3rd St & Pennsylvania Ave SE,31269,3rd St & Pennsylvania Ave SE,W22377,member,2019-10,NaN,2,38.887374,-77.001958,38.887375,-77.001955,38.887374,-77.001958,District of Columbia
2,563.0,2019-10-01 00:03:51,2019-10-01 00:13:14,31214,17th & Corcoran St NW,31251,12th & L St NW,W23473,member,2019-10,NaN,3,38.912097,-77.038689,38.903813,-77.028414,38.912097,-77.038689,District of Columbia
3,625.0,2019-10-01 00:05:34,2019-10-01 00:15:59,31301,Ward Circle / American University,31317,Wisconsin Ave & Ingomar St NW,W21371,member,2019-10,NaN,4,38.938741,-77.087181,38.958277,-77.084633,38.938741,-77.087181,District of Columbia
4,312.0,2019-10-01 00:05:47,2019-10-01 00:10:59,31519,1st & O St NW,31109,7th & T St NW,W23666,member,2019-10,NaN,5,38.908644,-77.012366,38.915499,-77.022204,38.908644,-77.012366,District of Columbia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5737813,886.0,2019-02-28 23:55:21,2019-03-01 00:10:08,31223,Convention Center / 7th & M St NW,31612,D St & Maryland Ave NE,W23129,member,2019-02,NaN,158126,38.905731,-77.022264,38.894761,-76.997124,38.905731,-77.022264,District of Columbia
5737814,404.0,2019-02-28 23:57:03,2019-03-01 00:03:47,31203,14th & Rhode Island Ave NW,31223,Convention Center / 7th & M St NW,W00316,member,2019-02,NaN,158127,38.908596,-77.032308,38.905737,-77.022268,38.908596,-77.032308,District of Columbia
5737815,972.0,2019-02-28 23:57:03,2019-03-01 00:13:16,31101,14th & V St NW,31600,5th & K St NW,W21796,member,2019-02,NaN,158128,38.917807,-77.032079,38.903043,-77.019032,38.917807,-77.032079,District of Columbia
5737816,925.0,2019-02-28 23:57:32,2019-03-01 00:12:58,31613,Eastern Market Metro / Pennsylvania Ave & 7th ...,31642,New Jersey Ave & L St NW,W23576,member,2019-02,NaN,158129,38.884090,-76.995180,38.902401,-77.016005,38.884090,-76.995180,District of Columbia


Merging two dataset resulted in a duplicated columns. So we are going to drop those and rename the columns.

In [36]:
# Dropping overlapping columns and renaming
df = df.drop(columns=['start_lat_y','start_lng_y'])
df = df.rename(columns = {'start_lat_x': 'start_lat', 'start_lng_x': 'start_lng'}, inplace = False)
df

,Duration,started_at,ended_at,start_station_id,start_station_name,end_station_id,end_station_name,ride_id,member_casual,fileid,rideable_type,id,start_lat,start_lng,end_lat,end_lng,state
0,429.0,2019-10-01 00:01:59,2019-10-01 00:09:08,31214,17th & Corcoran St NW,31203,14th & Rhode Island Ave NW,W23731,member,2019-10,NaN,1,38.912097,-77.038689,38.908600,-77.032307,District of Columbia
1,1935.0,2019-10-01 00:03:07,2019-10-01 00:35:23,31269,3rd St & Pennsylvania Ave SE,31269,3rd St & Pennsylvania Ave SE,W22377,member,2019-10,NaN,2,38.887374,-77.001958,38.887375,-77.001955,District of Columbia
2,563.0,2019-10-01 00:03:51,2019-10-01 00:13:14,31214,17th & Corcoran St NW,31251,12th & L St NW,W23473,member,2019-10,NaN,3,38.912097,-77.038689,38.903813,-77.028414,District of Columbia
3,625.0,2019-10-01 00:05:34,2019-10-01 00:15:59,31301,Ward Circle / American University,31317,Wisconsin Ave & Ingomar St NW,W21371,member,2019-10,NaN,4,38.938741,-77.087181,38.958277,-77.084633,District of Columbia
4,312.0,2019-10-01 00:05:47,2019-10-01 00:10:59,31519,1st & O St NW,31109,7th & T St NW,W23666,member,2019-10,NaN,5,38.908644,-77.012366,38.915499,-77.022204,District of Columbia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5737813,886.0,2019-02-28 23:55:21,2019-03-01 00:10:08,31223,Convention Center / 7th & M St NW,31612,D St & Maryland Ave NE,W23129,member,2019-02,NaN,158126,38.905731,-77.022264,38.894761,-76.997124,District of Columbia
5737814,404.0,2019-02-28 23:57:03,2019-03-01 00:03:47,31203,14th & Rhode Island Ave NW,31223,Convention Center / 7th & M St NW,W00316,member,2019-02,NaN,158127,38.908596,-77.032308,38.905737,-77.022268,District of Columbia
5737815,972.0,2019-02-28 23:57:03,2019-03-01 00:13:16,31101,14th & V St NW,31600,5th & K St NW,W21796,member,2019-02,NaN,158128,38.917807,-77.032079,38.903043,-77.019032,District of Columbia
5737816,925.0,2019-02-28 23:57:32,2019-03-01 00:12:58,31613,Eastern Market Metro / Pennsylvania Ave & 7th ...,31642,New Jersey Ave & L St NW,W23576,member,2019-02,NaN,158129,38.884090,-76.995180,38.902401,-77.016005,District of Columbia


We are doing with our geopy portion. Now moving on to adding COVID dataset.

# Adding COVID Dataset

## Cleaning up COVID dataset

We decided to take a dataset from Virginia department of Health, where they keep track of covid total confirmed cases counts, hospitalization counts and death counts for each counties and localities. 
Dataset is from here: https://data.virginia.gov/Government/VDH-COVID-19-PublicUseDataset-Cases/bre9-aqqr

In [37]:
# Bringing in the covid data
covid = pd.read_csv("/Users/jennyjang/Desktop/CS Project/VDH-COVID-19-PublicUseDataset-Cases.csv")
covid

,Report Date,FIPS,Locality,VDH Health District,Total Cases,Hospitalizations,Deaths
0,11/28/2020,51001,Accomack,Eastern Shore,"1,340",107,21
1,11/28/2020,51003,Albemarle,Thomas Jefferson,"1,896",100,27
2,11/28/2020,51005,Alleghany,Alleghany,316,19,7
3,11/28/2020,51007,Amelia,Piedmont,210,21,6
4,11/28/2020,51009,Amherst,Central Virginia,810,31,6
...,...,...,...,...,...,...,...
51865,04/10/2021,51800,Suffolk,Western Tidewater,"7,568",426,180
51866,04/10/2021,51810,Virginia Beach,Virginia Beach,"33,960","1,483",369
51867,04/10/2021,51820,Waynesboro,Central Shenandoah,"2,280",71,35
51868,04/10/2021,51830,Williamsburg,Peninsula,727,24,11


Since we want to add up the counts based on the date, we first need to get rid of the commas in the count column.

In [38]:
## convert the numbers to int. Don't rerun this as it has beeen done already. ##
covid['Total Cases'] =covid['Total Cases'].str.replace(',', '').astype(int)
covid['Hospitalizations'] =covid['Hospitalizations'].str.replace(',', '').astype(int)
covid['Deaths'] =covid['Deaths'].str.replace(',', '').astype(int)
covid

,Report Date,FIPS,Locality,VDH Health District,Total Cases,Hospitalizations,Deaths
0,11/28/2020,51001,Accomack,Eastern Shore,1340,107,21
1,11/28/2020,51003,Albemarle,Thomas Jefferson,1896,100,27
2,11/28/2020,51005,Alleghany,Alleghany,316,19,7
3,11/28/2020,51007,Amelia,Piedmont,210,21,6
4,11/28/2020,51009,Amherst,Central Virginia,810,31,6
...,...,...,...,...,...,...,...
51865,04/10/2021,51800,Suffolk,Western Tidewater,7568,426,180
51866,04/10/2021,51810,Virginia Beach,Virginia Beach,33960,1483,369
51867,04/10/2021,51820,Waynesboro,Central Shenandoah,2280,71,35
51868,04/10/2021,51830,Williamsburg,Peninsula,727,24,11


Taking the subset of df to perform aggregation and converting our report date format to datetime:

In [41]:
covid_temp = covid[['Report Date','Total Cases','Hospitalizations','Deaths']]
covid_temp['Report Date']= pd.to_datetime(covid_temp['Report Date'])
covid_temp

<ipython-input-41-da2f32e8d30b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_temp['Report Date']= pd.to_datetime(covid_temp['Report Date'])


,Report Date,Total Cases,Hospitalizations,Deaths
0,2020-11-28,1340,107,21
1,2020-11-28,1896,100,27
2,2020-11-28,316,19,7
3,2020-11-28,210,21,6
4,2020-11-28,810,31,6
...,...,...,...,...
51865,2021-04-10,7568,426,180
51866,2021-04-10,33960,1483,369
51867,2021-04-10,2280,71,35
51868,2021-04-10,727,24,11


Aggregating to add up the counts absed on the date:

In [42]:
covid_counts = covid_temp.groupby(['Report Date']).sum().sort_values('Report Date',ascending=False)
covid_counts=covid_counts.reset_index()
covid_counts

,Report Date,Total Cases,Hospitalizations,Deaths
0,2021-04-10,634325,27161,10458
1,2021-04-09,632625,27099,10451
2,2021-04-08,631083,27012,10436
3,2021-04-07,629155,26911,10415
4,2021-04-06,627605,26815,10401
...,...,...,...,...
385,2020-03-21,152,25,2
386,2020-03-20,114,20,2
387,2020-03-19,94,18,2
388,2020-03-18,77,17,2


We have successfully aggregated to get what we want. Now we have to merge this df to our big dataet based on the report date. To do that, we are extracting the date from our big dataset:

In [44]:
#Extracting date to use in merge with Covid df
df["started_at"] = pd.to_datetime(df['started_at'])
df['date'] = df['started_at'].dt.date

Now performing the perge:

In [48]:
# df["date"]= pd.to_datetime(df["date"])
# covid_counts["Report Date"]= pd.to_datetime(covid_counts['Report Date'])
# df = pd.merge(left=df, right=covid_counts, how='left', left_on='date', right_on='Report Date')
df

,Duration,started_at,ended_at,start_station_id,start_station_name,end_station_id,end_station_name,ride_id,member_casual,fileid,...,start_lat,start_lng,end_lat,end_lng,state,date,Report Date,Total Cases,Hospitalizations,Deaths
0,429.0,2019-10-01 00:01:59,2019-10-01 00:09:08,31214,17th & Corcoran St NW,31203,14th & Rhode Island Ave NW,W23731,member,2019-10,...,38.912097,-77.038689,38.908600,-77.032307,District of Columbia,2019-10-01,NaT,NaN,NaN,NaN
1,1935.0,2019-10-01 00:03:07,2019-10-01 00:35:23,31269,3rd St & Pennsylvania Ave SE,31269,3rd St & Pennsylvania Ave SE,W22377,member,2019-10,...,38.887374,-77.001958,38.887375,-77.001955,District of Columbia,2019-10-01,NaT,NaN,NaN,NaN
2,563.0,2019-10-01 00:03:51,2019-10-01 00:13:14,31214,17th & Corcoran St NW,31251,12th & L St NW,W23473,member,2019-10,...,38.912097,-77.038689,38.903813,-77.028414,District of Columbia,2019-10-01,NaT,NaN,NaN,NaN
3,625.0,2019-10-01 00:05:34,2019-10-01 00:15:59,31301,Ward Circle / American University,31317,Wisconsin Ave & Ingomar St NW,W21371,member,2019-10,...,38.938741,-77.087181,38.958277,-77.084633,District of Columbia,2019-10-01,NaT,NaN,NaN,NaN
4,312.0,2019-10-01 00:05:47,2019-10-01 00:10:59,31519,1st & O St NW,31109,7th & T St NW,W23666,member,2019-10,...,38.908644,-77.012366,38.915499,-77.022204,District of Columbia,2019-10-01,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5737813,886.0,2019-02-28 23:55:21,2019-03-01 00:10:08,31223,Convention Center / 7th & M St NW,31612,D St & Maryland Ave NE,W23129,member,2019-02,...,38.905731,-77.022264,38.894761,-76.997124,District of Columbia,2019-02-28,NaT,NaN,NaN,NaN
5737814,404.0,2019-02-28 23:57:03,2019-03-01 00:03:47,31203,14th & Rhode Island Ave NW,31223,Convention Center / 7th & M St NW,W00316,member,2019-02,...,38.908596,-77.032308,38.905737,-77.022268,District of Columbia,2019-02-28,NaT,NaN,NaN,NaN
5737815,972.0,2019-02-28 23:57:03,2019-03-01 00:13:16,31101,14th & V St NW,31600,5th & K St NW,W21796,member,2019-02,...,38.917807,-77.032079,38.903043,-77.019032,District of Columbia,2019-02-28,NaT,NaN,NaN,NaN
5737816,925.0,2019-02-28 23:57:32,2019-03-01 00:12:58,31613,Eastern Market Metro / Pennsylvania Ave & 7th ...,31642,New Jersey Ave & L St NW,W23576,member,2019-02,...,38.884090,-76.995180,38.902401,-77.016005,District of Columbia,2019-02-28,NaT,NaN,NaN,NaN


Replacing the NaNs to 0:

In [50]:
df['Total Cases'] = df['Total Cases'].replace(np.nan, 0)
df['Hospitalizations'] = df['Hospitalizations'].replace(np.nan, 0)
df['Deaths'] = df['Deaths'].replace(np.nan, 0)
df.sort_values(by='date')

,Duration,started_at,ended_at,start_station_id,start_station_name,end_station_id,end_station_name,ride_id,member_casual,fileid,...,start_lat,start_lng,end_lat,end_lng,state,date,Report Date,Total Cases,Hospitalizations,Deaths
5581774,531.0,2019-02-01 18:45:35,2019-02-01 18:54:27,31265,5th St & Massachusetts Ave NW,31519,1st & O St NW,W20094,member,2019-02,...,38.900928,-77.018678,38.908645,-77.012367,District of Columbia,2019-02-01,NaT,0.0,0.0,0.0
5581327,542.0,2019-02-01 17:05:10,2019-02-01 17:14:12,31201,15th & P St NW,31117,15th & Euclid St NW,W24170,member,2019-02,...,38.909853,-77.034412,38.923337,-77.035184,District of Columbia,2019-02-01,NaT,0.0,0.0,0.0
5581328,139.0,2019-02-01 17:05:27,2019-02-01 17:07:46,31257,22nd & I St NW / Foggy Bottom,31212,21st & M St NW,W23313,member,2019-02,...,38.900894,-77.048895,38.905708,-77.047290,District of Columbia,2019-02-01,NaT,0.0,0.0,0.0
5581329,697.0,2019-02-01 17:05:44,2019-02-01 17:17:22,31287,10th St & L'Enfant Plaza SW,31624,North Capitol St & F St NW,W00485,member,2019-02,...,38.884326,-77.026002,38.897455,-77.009879,District of Columbia,2019-02-01,NaT,0.0,0.0,0.0
5581330,450.0,2019-02-01 17:05:45,2019-02-01 17:13:15,31252,21st St & Pennsylvania Ave NW,31298,15th & M St NW,W00245,member,2019-02,...,38.901537,-77.046578,38.905422,-77.034672,District of Columbia,2019-02-01,NaT,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4980848,NaN,2021-03-31 09:40:34,2021-03-31 10:02:49,31666,4th St & G St SW,31127,22nd & H St NW,AE8DCA00BE5A93F7,member,2021-03,...,38.881118,-77.017656,38.898943,-77.048852,District of Columbia,2021-03-31,2021-03-31,618976.0,26455.0,10252.0
5097245,NaN,2021-03-31 12:26:16,2021-03-31 12:31:28,31248,Smithsonian-National Mall / Jefferson Dr & 12t...,31218,L'Enfant Plaza / 7th & C St SW,746307562A96591A,member,2021-03,...,38.888773,-77.028670,38.886266,-77.022243,District of Columbia,2021-03-31,2021-03-31,618976.0,26455.0,10252.0
4980847,NaN,2021-03-31 08:23:01,2021-03-31 08:26:41,31128,23rd & M St NW,31127,22nd & H St NW,6AC0B413F771BCDD,member,2021-03,...,38.905302,-77.050264,38.898943,-77.048852,District of Columbia,2021-03-31,2021-03-31,618976.0,26455.0,10252.0
4991152,NaN,2021-03-31 19:42:47,2021-03-31 20:35:30,31271,Constitution Ave & 2nd St NW/DOL,31638,1st & H St NW,7D55AC5D86E9625D,member,2021-03,...,38.892275,-77.013916,38.900370,-77.012108,District of Columbia,2021-03-31,2021-03-31,618976.0,26455.0,10252.0


We are now done with adding COVID data.

# Calculating duration of rides

Since we want to use the duration to compare the change in people's behavior, we are going to calculate duration. This column already exists in the dataset, but they are missing in the dataset from after covid. So we are going to calculate it. 

In [51]:
# Calculating duration to fill out missing values and converting into seconds
df["started_at"]=pd.to_datetime(df["started_at"])
df["ended_at"]=pd.to_datetime(df["ended_at"])
df["new_duration"] = df["ended_at"] - df["started_at"]
df['new_duration(min)'] = df['new_duration'].dt.total_seconds()
df.head()

,Duration,started_at,ended_at,start_station_id,start_station_name,end_station_id,end_station_name,ride_id,member_casual,fileid,...,end_lat,end_lng,state,date,Report Date,Total Cases,Hospitalizations,Deaths,new_duration,new_duration(min)
0,429.0,2019-10-01 00:01:59,2019-10-01 00:09:08,31214,17th & Corcoran St NW,31203,14th & Rhode Island Ave NW,W23731,member,2019-10,...,38.908600,-77.032307,District of Columbia,2019-10-01,NaT,0.0,0.0,0.0,0 days 00:07:09,429.0
1,1935.0,2019-10-01 00:03:07,2019-10-01 00:35:23,31269,3rd St & Pennsylvania Ave SE,31269,3rd St & Pennsylvania Ave SE,W22377,member,2019-10,...,38.887375,-77.001955,District of Columbia,2019-10-01,NaT,0.0,0.0,0.0,0 days 00:32:16,1936.0
2,563.0,2019-10-01 00:03:51,2019-10-01 00:13:14,31214,17th & Corcoran St NW,31251,12th & L St NW,W23473,member,2019-10,...,38.903813,-77.028414,District of Columbia,2019-10-01,NaT,0.0,0.0,0.0,0 days 00:09:23,563.0
3,625.0,2019-10-01 00:05:34,2019-10-01 00:15:59,31301,Ward Circle / American University,31317,Wisconsin Ave & Ingomar St NW,W21371,member,2019-10,...,38.958277,-77.084633,District of Columbia,2019-10-01,NaT,0.0,0.0,0.0,0 days 00:10:25,625.0
4,312.0,2019-10-01 00:05:47,2019-10-01 00:10:59,31519,1st & O St NW,31109,7th & T St NW,W23666,member,2019-10,...,38.915499,-77.022204,District of Columbia,2019-10-01,NaT,0.0,0.0,0.0,0 days 00:05:12,312.0


In [52]:
df.columns

Index(['Duration', 'started_at', 'ended_at', 'start_station_id',
       'start_station_name', 'end_station_id', 'end_station_name', 'ride_id',
       'member_casual', 'fileid', 'rideable_type', 'id', 'start_lat',
       'start_lng', 'end_lat', 'end_lng', 'state', 'date', 'Report Date',
       'Total Cases', 'Hospitalizations', 'Deaths', 'new_duration',
       'new_duration(min)'],
      dtype='object')

We are now done with adding duration. Since we are only interested in records that started from 'Virginia', going to subset our data to "state" == "Virginia".

In [54]:
virginia = df[df["state"]=="Virginia"]
virginia

,Duration,started_at,ended_at,start_station_id,start_station_name,end_station_id,end_station_name,ride_id,member_casual,fileid,...,end_lat,end_lng,state,date,Report Date,Total Cases,Hospitalizations,Deaths,new_duration,new_duration(min)
8,1904.0,2019-10-01 00:09:40,2019-10-01 00:41:25,31064,Gravelly Point,31612,D St & Maryland Ave NE,W21735,member,2019-10,...,38.894761,-76.997124,Virginia,2019-10-01,NaT,0.0,0.0,0.0,0 days 00:31:45,1905.0
10,777.0,2019-10-01 00:11:18,2019-10-01 00:24:15,31026,Washington Blvd & 10th St N,31073,Washington Blvd & Walter Reed Dr,W00410,member,2019-10,...,38.873223,-77.082106,Virginia,2019-10-01,NaT,0.0,0.0,0.0,0 days 00:12:57,777.0
36,1046.0,2019-10-01 01:07:52,2019-10-01 01:25:19,32220,Sunset Hills Rd & Isaac Newton Square,32216,Library St & Freedom Dr,W20489,casual,2019-10,...,38.959629,-77.358728,Virginia,2019-10-01,NaT,0.0,0.0,0.0,0 days 00:17:27,1047.0
45,1119.0,2019-10-01 01:53:24,2019-10-01 02:12:04,31014,Lynn St & 19th St North,31509,New Jersey Ave & R St NW,W22594,member,2019-10,...,38.912665,-77.017658,Virginia,2019-10-01,NaT,0.0,0.0,0.0,0 days 00:18:40,1120.0
47,1171.0,2019-10-01 01:56:47,2019-10-01 02:16:19,31013,Eads St & 22nd St S,31129,15th St & Pennsylvania Ave NW,W22874,member,2019-10,...,38.896466,-77.033377,Virginia,2019-10-01,NaT,0.0,0.0,0.0,0 days 00:19:32,1172.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5737718,883.0,2019-02-28 22:43:18,2019-02-28 22:58:01,31924,Wilson Blvd & N Troy St,31096,S Walter Reed Dr & 8th St S,W20183,member,2019-02,...,38.866127,-77.087869,Virginia,2019-02-28,NaT,0.0,0.0,0.0,0 days 00:14:43,883.0
5737735,181.0,2019-02-28 22:53:52,2019-02-28 22:56:53,31017,Wilson Blvd & N Uhle St,31029,N Veitch & 20th St N,W01318,member,2019-02,...,38.896928,-77.086484,Virginia,2019-02-28,NaT,0.0,0.0,0.0,0 days 00:03:01,181.0
5737762,584.0,2019-02-28 23:12:00,2019-02-28 23:21:44,31037,Ballston Metro / N Stuart & 9th St N,31020,Wilson Blvd & Franklin Rd,W22650,member,2019-02,...,38.890407,-77.088884,Virginia,2019-02-28,NaT,0.0,0.0,0.0,0 days 00:09:44,584.0
5737783,101.0,2019-02-28 23:27:28,2019-02-28 23:29:10,31031,15th & N Scott St,31018,N Rhodes & 16th St N,W01168,member,2019-02,...,38.892166,-77.079381,Virginia,2019-02-28,NaT,0.0,0.0,0.0,0 days 00:01:42,102.0


We are now done with our data cleaning. Saving our final dataset into csv file!

In [55]:
# virginia.to_csv(r'/Users/jennyjang/Desktop/CS Project/Virginia_records.csv')

Testing our data cleaning process. 
1. We create more specific exception references so that we know what went wrong specifically in our code. 
2. We test our data for: 
(a) presence of the key columns we need for analysis. 
(b) whether our key columns still have NaNs.  
(c) whether some of the columns have inappropriate data (for example negative duration).
2. We use unittest class to check that our assertions working and exceptions are raised as expected. 
(a) We feed incorrect code into our df.  
(b) Make sure our testing procedure picks up poor data.

In [ ]:
class ProjectTeamNANException(Exception): # so that we know what exactly went wrong, make exceptions more specific
    pass

In [ ]:
class ProjectTeamWrongValueException(Exception) # define our exception 
    pass

In [ ]:
def pre_analysis_check(df):
    df_temp = df4.copy()
    
    assert np.isin(['ride_id', 'start_lng', 'start_lat','new_duration(min)'], df_temp.columns).all() # columns exist
    
    if df_temp.loc[df_temp['start_lng'].isnull(), :].shape[0] > 0: # test if there are missing values in our columns (change columns)
        raise ProjectTeamNANException('Detected NANs in start_lng')   
         
    if df_temp.loc[df_temp['start_lat'].isnull(), :].shape[0] > 0: # test if there are missing values in our columns (change columns)
        raise ProjectTeamNANException('Detected NANs in start_lat')
        
    if df_temp.loc[df_temp['ride_id'].isnull(), :].shape[0] > 0: # test if there are missing values in our columns (change columns)
        raise ProjectTeamNANException('Detected NANs in ride_id')
        
    if df_temp.loc[df_temp['new_duration(min)'].isnull(), :].shape[0] > 0: # test if there are missing values in our columns (change columns)
        raise ProjectTeamNANException('Detected NANs in new_duration(min)')
        
    if df_temp.loc[df_temp['new_duration(min)']<0, :].shape[0] > 0: # test if there are missing values in our columns (change columns)
        raise ProjectTeamWrongValueException('Detected negative duration')
            
            

In [ ]:
import unittest
from pandas._testing import assert_frame_equal, assert_series_equal

class ProjectTeamTest(unittest.TestCase):
    def setUp(self):
        self.path_ = r'/Users/jennyjang/Desktop/CS Project/Virginia_records.csv' # path to working dir
                 
    def test_na(self):
        df = pd.read_csv(self.path_)
        df_temp.loc[:10, 'start_long'] = np.nan
        with self.assertRaises(ProjectTeamNANException): #check if our tests are working well 
            pre_analysis_check(df)
            
    def test_duration_negative(self):
        df = pd.read_csv(self.path_)
        df_temp = df.copy()    
        df_temp.loc[:10, 'duration'] = -10
        with self.assertRaises(ProjectTeamNANException): #check if our tests are working well 
            pre_analysis_check(df)
             
unittest.main(argv=[''], verbosity=2, exit=False)